# Convolutional Neural Net in Tensorflow & Keras
https://keras.io/
![Image](./data/02_network_flowchart.png)

---
---
# Keras Python Machine Learning API importieren

In [22]:
import keras
keras.__version__

'2.3.1'

---
---
# importieren der gesamten MNIST Datenbank

In [23]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### Ausgabe der Shape (Form)

In [24]:
print(train_images.shape)

(60000, 28, 28)


### Ausgabe der Dimension des Arrays mit `.nidm`

In [25]:
print(train_images.ndim)

3


---
---
# CNN erstellen

...im Prinzip wieder genauso wie bei einem `fully connected` Netz benötigen wir:

1. `Layer` 

2. `Eingabedaten` 

3. `Verlustfunktion`

4. `Optimierer`
<!--![Image](./data/nn.png)-->


## Module (layers & models) importieren

In [26]:
from keras import models
from keras import layers

## Initialisieren eines leeren Netzes
#### mit dem Modul `models` bestimmen wir im folgenden einen linearen Stapel von Schichten `(Sequential)`

In [27]:
cnn_model = models.Sequential()

##### Mit dem Argument `layers` definieren wir eine Liste von Layern, die dem Modell hinzugefügt werden sollen und legen die jewewiligen Attribute fest.

## 1. wir definieren einen Stapel von Conv2D- und MaxPooling-Layern

### 1.1.
`conv2d_1` Output > 32 Filter, bzw. Kanäle und jeder dieser Filter enthält ein 26x26 Raster mit Werten

In [28]:
#Layer (type)                 Output Shape              Param #   
#=================================================================
#conv2d_1 (Conv2D)            (None, 26, 26, 32)        320 
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

`input_shape` = 28 * 28 * Graustufenwert (image_height, image_with, image_channels)
![Image](./data/mnist-input.png)

+ Beispiel für eine Filteranwendung > `Filter` 3x3
![Image](./data/conv2d.png)

---
### 1.2.
`max_pooling2d_1` halbiert von 26x26 auf 13x13 und nimmt / extrahiert kleine Fenster daraus aus der Eingabe-Feature-Map, ausgehend von `conv2d_1` und liefert als Ausgabe dann die Maximalwerte weiter
<img src="./data/maxpooling.png" alt="drawing" width="600"/>

In [29]:
#Layer (type)                 Output Shape              Param #   
#=================================================================
#max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0   
cnn_model.add(layers.MaxPooling2D((2, 2)))

---
### 1.3.
`conv2d_2` Output > 64 Filter, bzw. Kanäle und jeder dieser Filter enthält ein 11x11 Raster mit Werten

In [30]:
#Layer (type)                 Output Shape              Param #   
#=================================================================
#conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

---
### 1.4.
`max_pooling2D`  halbiert von 11x11 auf 5x5 (durch padding kleiner) und nimmt / extrahiert kleine Fenster daraus aus der Eingabe-Feature-Map, ausgehend von `conv2d_2` und liefert als Ausgabe dann die Maximalwerte weiter

In [31]:
#Layer (type)                 Output Shape              Param #   
#=================================================================
#max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
cnn_model.add(layers.MaxPooling2D((2, 2)))

---
### 1.5.
`conv2d_3` Output > 64 Filter, bzw. Kanäle und jeder dieser Filter enthält ein 3x3 Raster mit Werten

In [32]:
#Layer (type)                 Output Shape              Param #   
#=================================================================
#conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))

## Darstellung der Convolutional Layer

In [33]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


---
---
## 2. nun definieren wir einen Stapel von Dense Layern als Klassifizierer:

In [34]:
#network.add(layers.Dense(10, activation='softmax'))
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(64, activation='relu'))
cnn_model.add(layers.Dense(10, activation='softmax'))

+ `Flatten` Vektorisiert die Shape auf 1-Dim, da Fully-Connected `Dense`-Layer nur eindimensionale Vektoren verarbeiten können
+ `64` bzw `10` = Anzahl der (fully-connected) Knoten
+ ...auf die letzten Endes die `softmax` funktion angewandt wird:

---
---

# Darstellung des gesamten Netzes

In [35]:
cnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

In [36]:
import pydot
import graphviz
import tensorflow as tf
tf.keras.utils.plot_model(cnn_model, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


# Trainings- bzw. Testdaten vorbereiten, bzw. skalieren:
<img src="./data/scale.png" width="600" />

In [37]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
#print("Datentyp before: ", test_images.dtype)
#print(test_images[1])
test_images = test_images.astype('float32') / 255
#print("Datentyp after: ", test_images.dtype)
#print(test_images[1])

---
---
# Kodieren der Klassen (Ziffern) von 0 bis 9 - mit One-Hot-Kodierung .:
<img src="./data/one-hot.png" width="300" />

In [38]:
from keras.utils import to_categorical

print("Beispiel Label vor der Encodierung: ", train_labels[1])
train_labels = to_categorical(train_labels)
print("Beilspiel Label nach der Encodierung: ", train_labels[1])

test_labels = to_categorical(test_labels)

Beispiel Label vor der Encodierung:  0
Beilspiel Label nach der Encodierung:  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


---
---
# kompilieren des Netzes

In [39]:
cnn_model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

+ `optimizer` Gewichtsanpassung https://keras.io/optimizers/ https://keras.io/optimizers/#rmsprop ...hier findet die Backpropagation (basierend auf Output der Verlustfunktion) statt

Verschiedene Optimierer können eines oder mehrere Konzepte (Algorithmen) nutzen, um die Effizienz des Gradientenabfalls bei einem bestimmten Trainingssatz zu verbessern - hier eine Liste von Gradientenabstiegsalgorithmen: http://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms

Ein Optimizer ist eine spezifische Implementierung des Gradientenabstiegs- Algorithmus. 


    Gewichtsaktualisierungsformel mathematisch ausgedrückt:
        delta_wjk = alpha * Ek * OK (1-OK) * OJT
        
            delta_wjk = Gewichtsänderung der Gewichte zwischen versteckter (j) und Ausgabeschicht (k)
            alpha     = Lernrate 
            EK        = Fehlerwerte der Ausgabeschicht << output_errors
            OK        = Ausgabewerte der Ausgabeschicht << final_outputs
            OjT       = Ausgabewerte der versteckten Schicht << hidden_outputs #was heißt T????
        
    Gewichtsaktualisierungsformel in Python:
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), 
                                        numpy.transpose(hidden_outputs))

            self.lr    = Lernrate
            numpy.dot()= Matrizenmultiplikation
            +=         = Addiere zum Wert self.who das Ergebnis des Ausdrucks auf der rechten Seite
            hidden_outputs = Ausgabewerte der versteckten Layer
            final_outputs  = Ausgabewerte des Outputlayers
            ouput_errors  = Fehlerwerte des Outputlayers 

+ `loss` Verlustfunktion E = S-K https://keras.io/losses/ https://keras.io/backend/#categorical_crossentropy
Verlustfunktion basierend auf dem absoluten Wert der Differenz zwischen den von einem Modell vorhergesagten Werten und den tatsächlichen Werten der Beschriftungen

  + Kreuzvalidierungsverfahren sind Testverfahren der Statistik bzw. der Datenanalyse


     Fehlerberechnung mathematisch ausgedrückt:
        E = S - K 

            Fehler = Sollwert - aktuellem Istwert

                       (  w11          w12  )
                       |-------     ------- |
                       |w11+w21     w12+w22 |   ( e1 )          ( w11   w12 )   ( e1 )
        error_hidden = |                    | * |    |     =    |           | * |    |
                       |  w21          w22  |   ( e2 )          (w21    w22 )   ( e2 )
                       |-------     ------- |
                       (w21+w11     w22+w12 )
        
        error_hidden = w_hidden-output         * error_output          

            hidden_errors = Gewichte zwischen Eingabe- und versteckter Schicht
            output_errors = Gewichte zwischen versteckter- und Ausgabeschicht   
                    output_errors = targets - final_outputs
                    targets = Optimalwerte aus der Testphase (Sollwert)

    Fehlerberechnung in Python:
        hidden_errors = numpy.dot(self.who.T, output_errors)      

+ `metrics`, definiert den Anteil der Bilder, die korrekt klassifiziert wurden https://keras.io/metrics/

---
---
# trainieren des Netzes
+ `fit()` übergibt dem Modell (NN) dee Numpy-Arrays der Eingabedaten `train_images` + Zielwerte `train_labels` 
+ `epochs` wie oft wir den gesamten Trainingssatz  durchlaufen möchten 
+ `batch_size`, Stapelgröße - wie viele Samples wir für eine Aktualisierung der Modellgewichte verwenden. 

In [40]:
cnn_model.fit(train_images, train_labels, epochs=7, batch_size=64)

Epoch 1/7
60000/60000 [==============================] - 6s 94us/step - loss: 0.1718 - accuracy: 0.9462
Epoch 2/7
60000/60000 [==============================] - 6s 93us/step - loss: 0.0465 - accuracy: 0.9855
Epoch 3/7
60000/60000 [==============================] - 6s 92us/step - loss: 0.0325 - accuracy: 0.9900
Epoch 4/7
60000/60000 [==============================] - 5s 92us/step - loss: 0.0252 - accuracy: 0.9924
Epoch 5/7
60000/60000 [==============================] - 5s 88us/step - loss: 0.0203 - accuracy: 0.9938
Epoch 6/7
60000/60000 [==============================] - 5s 91us/step - loss: 0.0156 - accuracy: 0.9954
Epoch 7/7
60000/60000 [==============================] - 5s 91us/step - loss: 0.0126 - accuracy: 0.9963


----
----
# Evaluation

In [41]:
test_loss, test_acc = cnn_model.evaluate(test_images, test_labels)
print('test_loss (Verlustrate):', test_loss)
print('test_acc (Korrektklassifzierungsrate):', test_acc)

10000/10000 [==============================] - 1s 82us/step
test_loss (Verlustrate): 0.028279399598229883
test_acc (Korrektklassifzierungsrate): 0.9922999739646912


Anzeige:
+ `loss` Verlustfunktion
+ `acc` accuracy, Genauigkeit der Trainningsdaten, Korrektklassifizierungsrate

---
---
# visuelle Evaluation...

In [45]:
import numpy as np
predictions = cnn_model.predict(test_images)
print("Vorhersage:")
print(predictions[17])
print(np.argmax(predictions[17]))
print(test_labels[17], "\n")

print("Originalbild aus der Datenbank:")
test_im = test_images[17]
plt.imshow(test_im.reshape(28,28), cmap='Greys')

Vorhersage:
[1.9838503e-15 4.9550327e-13 1.5146971e-13 1.2228893e-10 2.7463810e-12
 1.3398654e-13 3.2413297e-22 1.0000000e+00 1.5337268e-14 1.6315378e-11]
7
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] 

Originalbild aus der Datenbank:


AttributeError: module 'matplotlib' has no attribute 'imshow'

---
---
# Modell speichern

In [ ]:
# 11. Save model
cnn_model.save('mnist_trained_cnn_model.h5')  # creates a HDF5 file